# GPT Bargaining

In [ ]:
import sys
import os
import time


script_dir = os.path.abspath('../')
sys.path.append( script_dir )
print(sys.path)

In [ ]:
import openai
import re
import time
import json
import matplotlib.pyplot as plt


import numpy as np

from tqdm import tqdm
from pprint import pprint


from agent import DialogAgent, load_initial_instructions, involve_moderator, parse_final_price, BuyerAgent, SellerAgent, ModeratorAgent, SellerCriticAgent
from utils import wprint
from tenacity import retry, stop_after_attempt, wait_chain, wait_fixed

In [ ]:
#openai.api_key = "sk-" 

In [ ]:
# openai.organization = ""
# openai.api_key = "" 

In [ ]:
#Note: The openai-python library support for Azure OpenAI is in preview.
      #Note: This code sample requires OpenAI Python library version 0.28.1 or lower.
import os
import openai

openai.api_type = "azure"
openai.api_base = " "
openai.api_version = "2023-07-01-preview"
openai.api_key = ' '

message_text = [{"role":"system","content":"Now enter the role-playing mode. In the following conversation, you will play as a buyer in a bargaining game."},{"role":"user","content":"Let's play a game. You want to buy a balloon and you are bargaining with a seller. Your goal is to buy it with a low price, as low as possible. You should only respond to your seller with one short, succinct sentence. Now ask a price."},{"role":"assistant","content":"Hi, how much is the balloon?"},{"role":"user","content":"Hi, this is a good balloon and its price is $20"}]

completion = openai.ChatCompletion.create(
  engine="gpt-4",
  messages = message_text,
  temperature=0.8,
  max_tokens=800,
  top_p=0.95,
  frequency_penalty=1.5,
  presence_penalty=1.0,
  stop=None
)

print(completion['choices'][0]['message'])

# Bargaining

## Seller Init.

In [ ]:
seller_initial_dialog_history = load_initial_instructions('..\lib_prompt\seller.txt')
seller = SellerAgent(initial_dialog_history=seller_initial_dialog_history, agent_type="seller", engine="gpt-3.5-turbo")
pprint(seller.dialog_history)

In [ ]:
buyer_initial_dialog_history = load_initial_instructions('..\lib_prompt\\buyer.txt')
buyer = BuyerAgent(initial_dialog_history=buyer_initial_dialog_history, agent_type="buyer", engine="gpt-3.5-turbo")
pprint(buyer.dialog_history)

In [ ]:
moderator_initial_dialog_history = load_initial_instructions("..\lib_prompt\moderator_buyer.txt")
moderator = ModeratorAgent(initial_dialog_history=moderator_initial_dialog_history, agent_type="moderator", engine="gpt-3.5-turbo")
pprint(moderator.dialog_history)

In [ ]:
def run(buyer, seller, moderator, n=5, fd=None, verbose=True):
    wprint('seller: %s' % seller.last_response, fd, verbose=verbose)
    wprint('buyer: %s' % buyer.last_response, fd, verbose=verbose)
    
    wprint('---- start bargaining ----', fd, verbose=verbose)
    buyer_run = buyer.last_response
    start_involve_moderator = False
    deal_at = "none"
    for _ in range(n):
        time.sleep(1)
        seller_run = seller.call(buyer_run)
        wprint('seller: %s' % seller.last_response, fd, verbose=verbose)
        
        if(start_involve_moderator is False and involve_moderator(buyer_run, seller_run)):
            start_involve_moderator = True
            wprint('---- start moderating ----', fd, verbose=verbose)
        
        if(start_involve_moderator):
            time.sleep(1)
            moderate = moderator.moderate(seller.dialog_history,who_was_last="seller")
            wprint('MODERATE have the seller and the buyer achieved a deal? Yes or No: %s' % moderate, fd, verbose=verbose)
            if("yes" in moderate.lower()): 
                deal_at = "seller"
                break
            else: pass

        time.sleep(1)   
        buyer_run = buyer.call(seller_run)
        wprint('buyer: %s' % buyer.last_response, fd, verbose=verbose)
        
        if(start_involve_moderator is False and involve_moderator(buyer_run, seller_run)):
            start_involve_moderator = True
            wprint('---- start moderating ----', fd, verbose=verbose)
            
        if(start_involve_moderator):
            time.sleep(1)
            moderate = moderator.moderate( buyer.dialog_history, who_was_last="buyer")
            wprint('MODERATE have the seller and the buyer achieved a deal? Yes or No: %s' % moderate, fd, verbose=verbose)
            if("yes" in moderate.lower()): 
                deal_at = "buyer"
                break
            else: pass
                
    if(deal_at != "none"):
        if(deal_at == "seller"):
            final_price = parse_final_price(seller.dialog_history)
        else: 
            final_price = parse_final_price(buyer.dialog_history)
        return final_price
    else: return -1
    

In [ ]:
buyer.reset()
seller.reset()
moderator.reset()
final_price = run(buyer, seller, moderator, n=5)
print('FINAL PRICE: %s' % final_price)

# Initial Experiments, 100 runs, estimate final deal mean and variance

In [ ]:
final_prices = []
with open("outputs/initial_100_runs.txt", "w") as fd:
    for i in range(100):
        wprint("==== ROUND %d ====" % i, fd)
        buyer.reset()
        seller.reset()
        moderator.reset()
        final_price = run(buyer, seller, moderator, n=5, fd=fd)
        wprint('FINAL PRICE: %s' % final_price, fd)
        if(final_price != -1):
            final_prices.append(float(final_price))
        wprint("\n\n", fd)

In [ ]:
print(np.average(final_prices), np.std(final_prices))

In [ ]:
# Example list of data
data = np.array(final_prices)

# Divide data into 9 bins
bins = np.linspace(10, 20, 10)

# Compute histogram of the data
hist, bins = np.histogram(data, bins=bins)

# Create a bar plot using Matplotlib
plt.figure(figsize=(4, 2), dpi=250)
plt.bar(bins[:-1], hist, width=(bins[1]-bins[0]), align='edge')

# Set the title and axis labels
plt.title('Distribution of final price')
plt.xlabel('Final price')
plt.ylabel('Frequency')

# Display the plot
plt.show()

# AI critic coaching the seller

In [ ]:
def run_w_critic(buyer, seller, moderator, critic, n=5, fd=None, verbose=True):
    # Round 1
    wprint('==== ROUND 1 ====', fd, verbose=verbose)
    buyer.reset()
    seller.reset()
    moderator.reset()
    round_1_price = run(buyer, seller, moderator, n=10, fd=fd, verbose=verbose)
    wprint('PRICE: %s' % round_1_price, fd, verbose=verbose)
    
    # Round 2 after critic
    buyer.reset()
    moderator.reset()
    ai_feedback = critic.criticize(seller.dialog_history)
    wprint("FEEDBACK:\n%s\n\n" % ai_feedback, fd, verbose=verbose)
    acknowledgement = seller.receive_feedback(ai_feedback,round_1_price)
    
    wprint('==== ROUND 2 ====', fd, verbose=verbose)
    round_2_price = run(buyer, seller, moderator, n=10, fd=fd, verbose=verbose)
    wprint('PRICE: %s' % round_2_price, fd, verbose=verbose)
    
    # Round 3 after critic
#     buyer.reset()
#     moderator.reset()
#     ai_feedback = critic.criticize(seller.dialog_history)
#     wprint("FEEDBACK: %s\n\n" % ai_feedback, fd)
#     acknowledgement = seller.receive_feedback(ai_feedback)
    
#     wprint('==== ROUND 3 ====', fd)
#     price_after_feedback = run(buyer, seller, moderator, n=10, fd=fd)
#     wprint('PRICE: %s' % price_after_feedback, fd)
    return round_1_price, round_2_price

In [ ]:
critic_initial_dialog_history = load_initial_instructions("..\lib_prompt\seller_critic.txt")
critic = SellerCriticAgent(initial_dialog_history=critic_initial_dialog_history, agent_type="critic", engine="gpt-3.5-turbo")
pprint(critic.dialog_history)

In [ ]:
run_w_critic(buyer, seller, moderator, critic, n=5, fd=None)

In [ ]:
round_1_prices = []
round_2_prices = []
verbose = False
with open("outputs/seller_critic_100_runs.txt", "w") as fd:
    for i in tqdm(range(100)):
        wprint("==== CASE %d ====" % i, fd, verbose=verbose)
        buyer.reset()
        seller.reset()
        moderator.reset()
        round_1_price, round_2_price = run_w_critic(buyer, seller, moderator, critic, n=8, fd=fd, verbose=False)
        
        if(round_1_price > 10 and round_1_price < 20 and round_2_price > 10 and round_2_price < 20):
            round_1_prices.append(round_1_price)
            round_2_prices.append(round_2_price)
        wprint("\n\n\n\n", fd, verbose=verbose)

In [ ]:
round_2_price

In [ ]:
print(np.average(round_1_prices), np.std(round_1_prices))

In [ ]:
print(np.average(round_2_prices), np.std(round_2_prices))

In [ ]:
# Example list of data
data_1 = np.array(round_1_prices)
data_2 = np.array(round_2_prices)

# Divide data into 9 bins
bins_1 = np.linspace(10, 20, 10)
bins_2 = np.linspace(10, 20, 10)

# Compute histogram of the data
hist_1, bins_1 = np.histogram(data_1, bins=bins_1)
hist_2, bins_2 = np.histogram(data_2, bins=bins_2)

# Create a bar plot using Matplotlib
plt.figure(figsize=(4, 2), dpi=250)
plt.bar(bins_1[:-1]+0, hist_1, width=0.3 * (bins_1[1]-bins_1[0]), align='edge', label='Before AI Critic')
plt.bar(bins_2[:-1]+0.3, hist_2, width=0.3 * (bins_2[1]-bins_2[0]), align='edge', label='After AI Critic')

# Set the title and axis labels
plt.title('Distribution of deal price')
xticks = np.arange(10, 21, 2)
plt.xticks(xticks, [str(x) for x in xticks], fontsize=8)
plt.xlabel('Deal price')
plt.ylabel('Frequency')
plt.legend(fontsize=7)

# Display the plot
plt.show()